In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import mutual_info_classif
import json

from mepylome import CNV, MethylData, ReferenceMethylData
from mepylome.analysis import MethylAnalysis
import os

In [2]:
import json

with open('data/EPIC_v1_EPIC_v2_common_features.csv') as f:
    EPIC_v1_EPIC_v2_common_features = json.load(f)

In [3]:
def get_matching_filename(basename, idat_dir):
    for file in os.listdir(idat_dir):
        if basename in file:
            return file.replace('_Grn.idat', '').replace('_Red.idat', '')
    return basename


# Extract beta values from idat

In [4]:
data_anno = pd.read_csv('data/all_sample_anno.csv')
idat_dir_EPIC = 'data/EPIC/'
idat_dir_EPICv2 = 'data/EPICv2/'

# Filter for EPIC in column Platform
anno_epic = data_anno[data_anno['Platform'] == 'EPIC']
anno_epic_v2 = data_anno[data_anno['Platform'] == 'EPICv2']

In [6]:
# Load EPIC list
basename_epic = anno_epic['Basename'].tolist()
list_epic = []
for i in basename_epic:
    list_epic.append(f'{idat_dir_EPIC}/{get_matching_filename(i, idat_dir_EPIC)}')

# Load EPIC betas
betas_epic = MethylData(file=list_epic, prep="noob").betas.loc[EPIC_v1_EPIC_v2_common_features]


In [5]:
# Load EPIC list
basename_epic_v2 = anno_epic_v2['Basename'].tolist()
list_epic_v2 = []
for i in basename_epic_v2:
    list_epic_v2.append(f'{idat_dir_EPICv2}/{get_matching_filename(i, idat_dir_EPICv2)}')

# Load EPIC betas
betas_epic_v2 = MethylData(file=list_epic_v2, prep="noob").betas.loc[EPIC_v1_EPIC_v2_common_features]


In [14]:
# remove duplicate index values
betas_epic = betas_epic.loc[~betas_epic.index.duplicated(keep='first')]
betas_epic_v2 = betas_epic_v2.loc[~betas_epic_v2.index.duplicated(keep='first')]


In [23]:
# Combine the EPIC and EPICv2 betas keeping common rows
all_beta_values = pd.concat([betas_epic, betas_epic_v2], axis=1, join='inner')
all_beta_values = all_beta_values.loc[~all_beta_values.index.duplicated(keep='first')]
all_beta_values.shape

(671727, 114)

In [25]:
# Save to file
all_beta_values.to_csv('data/all_beta_values.csv')

# Get Features

In [29]:
# Load annotation
step_1_anno = pd.read_csv('data/step_1_training_anno.csv', index_col=0)
step_2_anno = pd.read_csv('data/step_2_training_anno.csv', index_col=0)
step_3_anno = pd.read_csv('data/step_3_training_anno.csv', index_col=0)
testing_anno = pd.read_csv('data/testing_set_anno.csv', index_col=0)

# Filter betas
step_1_betas = all_beta_values[step_1_anno['Sample_Name'].tolist()]
step_2_betas = all_beta_values[step_2_anno['Sample_Name'].tolist()]
step_3_betas = all_beta_values[step_3_anno['Sample_Name'].tolist()]
testing_betas = all_beta_values[testing_anno['Sample_Name'].tolist()]

(671727, 114)

In [ ]:
# keep only top 5000 rows with highest std
def filter_top_std(betas, n):
    betas['row_std'] = betas.std(axis=1)
    betas = betas.sort_values(by='row_std', ascending=False)
    betas = betas.head(n)
    return betas

def filter_high_corr(betas, corr_threshold, label):
    # Compute Mutual Information
    mi_scores = mutual_info_classif(betas, label, random_state=42)
    mi_values = {}
    for i in range(len(betas.columns.tolist())):
        mi_values[betas.columns.tolist()[i]] = mi_scores[i]
        i += 1

    # Filter features with high correlation to each other
    correlation_matrix = betas.corr().abs()

    # Find feature pairs with correlation above 0.9
    high_corr_pairs = np.where((np.abs(correlation_matrix) > corr_threshold) & (np.abs(correlation_matrix) < 1.0))

    # Create a set to track features to drop
    features_to_drop = set()

    # Iterate through pairs and drop the feature with lower standard deviation
    for i, j in zip(*high_corr_pairs):
        feature1 = correlation_matrix.columns[i]
        feature2 = correlation_matrix.columns[j]
        if feature1 not in features_to_drop and feature2 not in features_to_drop:
            # Compare standard deviations and drop the lower one
            if mi_values[feature1] > mi_values[feature2]:
                features_to_drop.add(feature2)
            else:
                features_to_drop.add(feature1)

    # Drop features
    betas_filtered = betas.drop(columns=features_to_drop)

    return betas_filtered

In [ ]:
# filter top 5000 rows with highest std
top_5000_std_step_1 = filter_top_std(step_1_betas, 5000)
top_5000_std_step_2 = filter_top_std(step_2_betas, 5000)
top_5000_std_step_3 = filter_top_std(step_3_betas, 5000)

# filter high correlation
top_5000_std_step_1_filtered = filter_high_corr(top_5000_std_step_1, 0.9, step_1_anno['Label_PMOC'])
top_5000_std_step_2_filtered = filter_high_corr(top_5000_std_step_2, 0.9, step_2_anno['Label_SMOC'])
top_5000_std_step_3_filtered = filter_high_corr(top_5000_std_step_3, 0.9, step_3_anno['Label_STAD_COAD'])

testing_betas_filtered_step_1 = testing_betas.loc[top_5000_std_step_1_filtered.index.tolist()]
testing_betas_filtered_step_2 = testing_betas.loc[top_5000_std_step_2_filtered.index.tolist()]
testing_betas_filtered_step_3 = testing_betas.loc[top_5000_std_step_3_filtered.index.tolist()]

# extract features
feature_cpg_list_step_1 = top_5000_std_step_1_filtered.index.tolist()
feature_cpg_list_step_2 = top_5000_std_step_2_filtered.index.tolist()
feature_cpg_list_step_3 = top_5000_std_step_3_filtered.index.tolist()

# Save to file
top_5000_std_step_1_filtered.to_csv('data/top_5000_std_step_1_filtered.csv')
top_5000_std_step_2_filtered.to_csv('data/top_5000_std_step_2_filtered.csv')
top_5000_std_step_3_filtered.to_csv('data/top_5000_std_step_3_filtered.csv')

testing_betas_filtered_step_1.to_csv('data/testing_betas_filtered_step_1.csv')
testing_betas_filtered_step_2.to_csv('data/testing_betas_filtered_step_2.csv')
testing_betas_filtered_step_3.to_csv('data/testing_betas_filtered_step_3.csv')

with open('data/feature_cpg_list_step_1.json', 'wb') as f:
    json.dump(feature_cpg_list_step_1, f)

with open('data/feature_cpg_list_step_2.json', 'wb') as f:
    json.dump(feature_cpg_list_step_2, f)

with open('data/feature_cpg_list_step_3.json', 'wb') as f:
    json.dump(feature_cpg_list_step_3, f)